In [5]:
import pandas as pd
prices = pd.read_csv("prices.csv")
print(prices.sort_values(['product_id', 'updated_at']))

    product_id  old_price  new_price     updated_at
0           64     270000     239000  9/10/18 16:37
4      3954203      68800      60000  9/10/18 16:32
1      3954203      60000      64000  9/11/18 11:54
3      3954203      64000      60500   9/15/18 3:49
2      3954203      60500      57500  9/17/18 22:59
7      3998909      19000      17000  9/10/18 16:35
6      3998909      17000      15500   9/13/18 6:43
5      3998909      15500      16500   9/16/18 5:09
11     4085861      60000      53500   9/11/18 8:51
9      4085861      53500      67000   9/12/18 3:51
13     4085861      67000      62500   9/13/18 6:43
12     4085861      62500      58000   9/15/18 3:51
8      4085861      53500      52000  9/17/18 22:59
10     4085861      58000      53500   9/17/18 3:35


In [6]:
sales = pd.read_csv("sales.csv")
print(sales.sort_values(['product_id', 'ordered_at']))

     product_id     ordered_at  quantity_ordered
171          64  9/11/18 19:32                 1
174          64  9/12/18 13:23                 1
173          64  9/12/18 16:00                 1
172          64  9/14/18 20:10                 1
24      3954203  9/11/18 11:11                 1
..          ...            ...               ...
79      4085861  9/18/18 20:54                 1
69      4085861   9/18/18 6:47                 1
68      4085861   9/18/18 6:53                 1
74      4085861   9/18/18 7:01                 1
70      4085861   9/18/18 8:29                 1

[175 rows x 3 columns]


In [26]:
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
prices = prices.sort_values('updated_at')
sales = sales.sort_values('ordered_at')
merged_sale_price = pd.merge_asof(sales, prices, left_on = 'ordered_at', right_on = 'updated_at', by = 'product_id', direction='nearest' )
print(merged_sale_price)

     product_id          ordered_at  quantity_ordered  old_price  new_price  \
0       3954203 2018-09-11 01:43:00                 1      68800      60000   
1       4085861 2018-09-11 06:26:00                 1      60000      53500   
2       4085861 2018-09-11 06:53:00                 1      60000      53500   
3       4085861 2018-09-11 08:24:00                 1      60000      53500   
4       4085861 2018-09-11 09:30:00                 1      60000      53500   
..          ...                 ...               ...        ...        ...   
170     4085861 2018-09-18 20:23:00                 1      53500      52000   
171     4085861 2018-09-18 20:43:00                 1      53500      52000   
172     4085861 2018-09-18 20:54:00                 1      53500      52000   
173     3954203 2018-09-18 21:26:00                 1      60500      57500   
174     3998909 2018-09-18 22:11:00                 1      15500      16500   

             updated_at  
0   2018-09-10 16:32:00  

In [27]:
merged_sale_price['final_price']=merged_sale_price.apply(lambda row: row['new_price'] if row['ordered_at'] >= row['updated_at'] else row['old_price'], axis=1)
merged_sale_price['revenue']=merged_sale_price['quantity_ordered']*merged_sale_price['final_price']
data=merged_sale_price.loc[:,['product_id', 'revenue']]
print(data)

     product_id  revenue
0       3954203    60000
1       4085861    60000
2       4085861    60000
3       4085861    60000
4       4085861    53500
..          ...      ...
170     4085861    52000
171     4085861    52000
172     4085861    52000
173     3954203    57500
174     3998909    16500

[175 rows x 2 columns]


In [28]:
final_data=data.groupby('product_id').sum('revenue')
print(final_data)

            revenue
product_id         
64           956000
3954203      877500
3998909      280500
4085861     8247500
